In [ ]:
!pip install transformers datasets

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset
data_path = '/content/drive/MyDrive/email_spam.csv'
df = pd.read_csv(data_path)

# Map 'spam' to 1 and 'not spam' to 0
df['type'] = df['type'].map({'spam': 1, 'not spam': 0})

In [ ]:
# Split the dataset into training and testing
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['type'])

# Initialize the tokenizer and model
tokenizer_spam = BertTokenizer.from_pretrained('bert-base-uncased')
model_spam = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the datasets
def spam_tokenize_function(examples):
    return tokenizer_spam(examples['text'], padding="max_length", truncation=True, max_length=128)

train_dataset_spam = Dataset.from_pandas(train_df).map(spam_tokenize_function, batched=True)
test_dataset_spam = Dataset.from_pandas(test_df).map(spam_tokenize_function, batched=True)

train_dataset_spam = train_dataset_spam.rename_column('type', 'labels')
test_dataset_spam = test_dataset_spam.rename_column('type', 'labels')

train_dataset_spam.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset_spam.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/67 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args_spam = TrainingArguments(
    output_dir='./results_spam',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs_spam',
    logging_steps=10,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy='epoch',
    load_best_model_at_end=True,
    seed=42,
)

# Metrics for evaluation
def spam_compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc}

# Initialize Trainer
trainer_spam = Trainer(
    model=model_spam,
    args=training_args_spam,
    train_dataset=train_dataset_spam,
    eval_dataset=test_dataset_spam,
    compute_metrics=spam_compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train and evaluate the model
trainer_spam.train()
eval_results_spam = trainer_spam.evaluate()
print(f"Spam Classification Evaluation results: {eval_results_spam}")

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.568588,0.705882
2,0.574100,0.557311,0.705882
3,0.574100,0.550239,0.705882
4,0.455100,0.530881,0.705882


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: Undefin

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.568588,0.705882
2,0.574100,0.557311,0.705882
3,0.574100,0.550239,0.705882
4,0.455100,0.530881,0.705882
5,0.455100,0.529938,0.705882


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Spam Classification Evaluation results: {'eval_loss': 0.5299381017684937, 'eval_accuracy': 0.7058823529411765, 'eval_runtime': 6.0548, 'eval_samples_per_second': 2.808, 'eval_steps_per_second': 0.33, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Predict and show examples
predictions_spam = trainer_spam.predict(test_dataset_spam)
preds_spam = np.argmax(predictions_spam.predictions, axis=-1)



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Print examples of predictions
for i, (prediction, label) in enumerate(zip(preds_spam[:10], test_df['type'].values[:10])):
    text = test_df['text'].iloc[i]
    pred_label = 'Spam' if prediction == 1 else 'Not Spam'
    true_label = 'Spam' if label == 1 else 'Not Spam'
    print(f"Prediction: {pred_label}, True Label: {true_label}\nText: {text}\n")



Prediction: Not Spam, True Label: Not Spam
Text: TOP STORIES FOR VIRA

Quora
?
If coding in C++ is so extremely hard, why do we have so many PlayStation and Nintendo video games and many computer programs that are coded in C++ if it is so hard for our brains to learn?
Andrew McGregor, Performance Measurement Lead at Fastly (2019-present) • Answered January 9, 2021

C++ is moderately hard, it is true. But making a high-end game is really, really hard. Much harder than any programming language. With any large application, the problem is t... Read more »
Upvote 
1.6K

Comment 
55

?
They say C is an unsafe language. As a beginner in C programming (say I only have experience in Python), what are some things I should avoid when using C, such as bad habits or dangerous code/syntax?
Asked in C Programmers • Answered by Manuel Mateo • April 2

Always set pointers to null after freeing them
Always check if pointers are null
Prefer the number-capped versions of functions (strncpy vs strcpy and o